# Hola, mundo en LangChain

## Instalar librerías principales y configuración de API Key de OpenAI

In [5]:
%%capture
!pip install langchain pypdf openai chromadb tiktoken
#pypdf ayuda a cargar documentos pfd de manera local luego se coviertene n documentos que sea legibles con lanchain
#openia se usará un modelo de lenguaje de ipen ia
#chromadb es una base de datos bectorial se usa pararear indices y encontrar documentos que más sirven para responder la pregunta al usuario
#tiktoken esta librería sirve o se usa para tokenizar u obtener información de los tokens que se ingresan 

#

In [8]:
!pip install -U langchain-community

  Obtaining dependency information for langchain-community from https://files.pythonhosted.org/packages/d2/27/9c310c60c572b69a8eeb27f828b0df097834062862f541128b02b87df8f0/langchain_community-0.2.5-py3-none-any.whl.metadata
  Using cached langchain_community-0.2.5-py3-none-any.whl.metadata (2.5 kB)
  Obtaining dependency information for dataclasses-json<0.7,>=0.5.7 from https://files.pythonhosted.org/packages/c3/be/d0d44e092656fe7a06b55e6103cbce807cdbdee17884a5367c68c9860853/dataclasses_json-0.6.7-py3-none-any.whl.metadata
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Obtaining dependency information for marshmallow<4.0.0,>=3.18.0 from https://files.pythonhosted.org/packages/96/d7/f318261e6ccbba86bdf626e07cd850981508fdaec52cfcdc4ac1030327ab/marshmallow-3.21.3-py3-none-any.whl.metadata
  Using cached marshmallow-3.21.3-py3-none-any.whl.metadata (7.1 kB)
  Obtaining dependency information for typing-inspect<1,>=0.4.0 from https://files.pythonhosted.org/packages


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


para obtener la la apikey se debe ingresar a chatgpt, a tu perfil, y all+i aparecera user Api keys, aparecerá una lista o la creación de un nuevo tokken allí se le dad´ra un nombre al token y posterioemente se le dará guardar una vez se guarde se debe esperar a que entregue la apikey del usuario


In [20]:
from getpass import getpass
import os

OPENAI_API_KEY = getpass('Enter the secret value: ')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
#aqui se carga la apikey de GPT

## Carga de documents

In [10]:
import requests
from langchain.document_loaders import PyPDFLoader
#carga de documentos pdf mediante links 
urls = [
    'https://arxiv.org/pdf/2306.06031v1.pdf',
    'https://arxiv.org/pdf/2306.12156v1.pdf',
    'https://arxiv.org/pdf/2306.14289v1.pdf',
    'https://arxiv.org/pdf/2305.10973v1.pdf',
    'https://arxiv.org/pdf/2306.13643v1.pdf'
]

ml_papers = []

for i, url in enumerate(urls):
    response = requests.get(url)
    filename = f'paper{i+1}.pdf'
    with open(filename, 'wb') as f:
        f.write(response.content)
        print(f'Descargado {filename}')
        #descarga de documentos y modificacioens para que langchain pueda interpretarlos
        loader = PyPDFLoader(filename)#se crea loader sdonde se pone el pypdf para car gar la información que se quiere en este caso el nombre del documento que se crea en filename
        data = loader.load()#el loader tiene la función que se llama load que es un pdf original mente y lo convierte en un documet esta es la clase que sera usada por langchain para procesar la información
        ml_papers.extend(data)#se guarda la información que se tiene para crear una lista donde cada elemento sea una parte de los PDF




# Utiliza la lista ml_papers para acceder a los elementos de todos los documentos descargados
print('Contenido de ml_papers:')
print()

Descargado paper1.pdf
Descargado paper2.pdf
Descargado paper3.pdf
Descargado paper4.pdf
Descargado paper5.pdf
Contenido de ml_papers:



In [11]:
type(ml_papers), len(ml_papers), ml_papers[3]
#ver como luce el PDF

(list,
 57,
 Document(page_content='Figure 1: FinGPT Framework.\n4.1 Data Sources\nThe first stage of the FinGPT pipeline involves the collec-\ntion of extensive financial data from a wide array of online\nsources. These include, but are not limited to:\n•Financial news: Websites such as Reuters, CNBC, Yahoo\nFinance, among others, are rich sources of financial news\nand market updates. These sites provide valuable informa-\ntion on market trends, company earnings, macroeconomic\nindicators, and other financial events.\n•Social media : Platforms such as Twitter, Facebook, Red-\ndit, Weibo, and others, offer a wealth of information in\nterms of public sentiment, trending topics, and immediate\nreactions to financial news and events.\n•Filings : Websites of financial regulatory authorities, such\nas the SEC in the United States, offer access to company\nfilings. These filings include annual reports, quarterly earn-\nings, insider trading reports, and other important company-\nspecific in

## Split de documents

In [12]:
#convertir los documentos en números 
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,#cada texto tendrá un tamaño de 1500
    chunk_overlap=200,#al principio tendra 200 caracteres se que repiten en el fragmento pasado y al final tendra 200 que se repetiran
    length_function=len #se pone len ya qu ees la longitud de caracteres
    )

documents = text_splitter.split_documents(ml_papers)
#se parten los mlpappers y se convierten en documentos más pequeños

In [15]:
len(documents), documents[10]
#se revisan los documentos como se han partido

(211,
 Document(page_content='highly volatile, changing rapidly in response to news events\nor market movements.\nTrends , often observable through websites like Seeking\nAlpha, Google Trends, and other finance-oriented blogs and\nforums, offer critical insights into market movements and in-\nvestment strategies. They feature:\n•Analyst perspectives: These platforms provide access to\nmarket predictions and investment advice from seasoned\nfinancial analysts and experts.\n•Market sentiment: The discourse on these platforms can\nreflect the collective sentiment about specific securities,\nsectors, or the overall market, providing valuable insights\ninto the prevailing market mood.\n•Broad coverage: Trends data spans diverse securities and\nmarket segments, offering comprehensive market coverage.\nEach of these data sources provides unique insights into\nthe financial world. By integrating these diverse data types,\nfinancial language models like FinGPT can facilitate a com-\nprehensive 

## Embeddings e ingesta a base de datos vectorial

In [24]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
#con el texto se puede alimentar un modelo embeddings todo se pone en una base da datos vectorial 
# por eso se usa chroma 

embeddings = OpenAIEmbeddings(model ="text-embedding-ada-002") #que embeddings se va a usar o modelo
#el modelo es el que se puede cambiar
vectorstore = Chroma.from_documents(
    documents = documents,# se le da que documentos tendra convertidos después en números  estos son los documentos anteriores
    embedding = embeddings #el modelo a usar el donde esta incrustación
) #base de datos vectorial vector store es la base de datos vectorial  

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 1} # entre más se ponga más se va a gastar enel promp parametro de la busqueda
    )#retriver loq ue hace es que convierte la base de datos en algo que puede buscar la información relevante va a devolver fragmentos clave para resolver una pregúnta 

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

## Modelos de chat y cadenas para consulta de información

In [25]:
#aqui se define el modelo lenguaje o el modelo de chat a usar para resolver las dudas
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
#se crea una cadena que resuelve pregúntas 

chat = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo',#nombre del modelo a usar en este caso el GPT 3.5
    temperature=0.0 #recordar que la temperatura va de 0 a 1 donde 0 es algo preciso y 1 es que puede decir cualquier mamada o algo creativo 
)

qa_chain = RetrievalQA.from_chain_type(
    llm= chat,#se le indica que modelo a usar 
    chain_type="stuff",#cadena de tipo stuff que es loque quepa en el prompt
    retriever=retriever
)#aqui se crea la cadena donde QA es question ansering es decir resolver pregúntas  

NameError: name 'retriever' is not defined

In [26]:
query = "qué es fingpt?"
qa_chain.run(query)

NameError: name 'qa_chain' is not defined

In [ ]:
query = "qué hace complicado entrenar un modelo como el fingpt?"
qa_chain.run(query)

In [ ]:
query = "qué es fast segment?"
qa_chain.run(query)

In [ ]:
query = "cuál es la diferencia entre fast sam y mobile sam?"
qa_chain.run(query)